# Imports and data preparation

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from pathlib import Path
from datetime import datetime
import os

from openstef.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test
from openstef.metrics.figure import plot_feature_importance
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass

# Set working dir to location of this file
os.chdir('.')

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = 'plotly'

In [ ]:
import plotly.io as pio

# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"

## EMS measurements
Load, pre-process, and visualize 

In [ ]:
# Load inputs
filename = Path("../.data/Middenmeer-150kV.csv")

measurements = pd.read_csv(filename, delimiter=";", decimal=",")
measurements["Datetime"] = pd.to_datetime(measurements["Datum"] + " " + measurements["Tijd"])
measurements = measurements.set_index('Datetime').tz_localize('CET', ambiguous='NaT', nonexistent='NaT').tz_convert("UTC")

# Only keep relevant columns
measurements = measurements.iloc[:,2:-1]

# Sum the load
measurements['Total'] = measurements.sum(axis=1)

# By default, only a backtest is made for the total
target_column = 'Total'

measurements.plot()

### Check the validity of the measurements

In [ ]:
# Show all rows with a duplicate index
measurements[measurements.index.duplicated(keep=False)]

In [ ]:
# Drop all rows with a NaT index.
measurements = measurements[measurements.index.notna()]

In [ ]:
# Validate that there are no duplicates left
assert not(measurements.index.duplicated().any()), "Duplicate indices have been found in the measurements dataframe."

## C-ARM & T-prognoses Vattenfall
Load, pre-process, and visualize  

In [ ]:
large_clients = pd.read_csv("../.data/middenmeer_clients.csv", index_col=0)
vattenfall_mrids = large_clients.query("'Vattenfall Windpark Wieringermeer B.V.' in Name")["mRID"]

carm_measurements = pd.read_csv("../.data/mdm_customer_carm_measurements.csv", 
                                delimiter=",", decimal=".", index_col=0, parse_dates=True)
customer_power_forecast = pd.read_csv("../.data/mdm_customer_power_forecasts_2022.csv", 
                             delimiter=",", decimal=".", index_col=0, parse_dates=True)

In [ ]:
vattenfall_carm_measurements = carm_measurements[vattenfall_mrids]
vattenfall_power_forecasts = customer_power_forecast[vattenfall_mrids]

In [ ]:
aggregated_vattenfall_load_watt = vattenfall_carm_measurements.sum(axis=1)
aggregated_vattenfall_power_forecasts_watt = vattenfall_power_forecasts.sum(axis=1)

In [ ]:
aggregated_vattenfall_load = aggregated_vattenfall_load_watt / 1000000
aggregated_vattenfall_power_forecasts = aggregated_vattenfall_power_forecasts_watt / 1000000

## Predictors
Load, pre-process, and visualize 

In [ ]:
# Load predictors
predictors = pd.read_csv('../.data/weather_apx_sji_sja_Middenmeer.csv', index_col=0, parse_dates=True)
predictors.head()

In [ ]:
# Check the validity of the predictors data
assert not(predictors.duplicated().any()), "Duplicate values have been found in the predictors dataframe."
assert not(predictors.index.duplicated().any()), "Duplicate indices have been found in the predictors dataframe."

# Backtests

## Top down Middenmeer forecasts

### Combine EMS measurements and predictors to get input data

In [ ]:
# OpenSTEF always expects a column called "load". This is the column it will predict.
load = pd.DataFrame(dict(load=measurements.loc[:,target_column]))
input_data = load.merge(predictors, left_index=True, right_index=True, how='inner')

In [ ]:
assert not(input_data.index.duplicated().any()), "There are duplicate indices in the input data."

### Backtest configuration and execution

#### Configure training, prediction, and backtest specifications

In [ ]:
# Define properties of training / prediction. We call this a 'prediction_job'.
pj=PredictionJobDataClass(
    id=1, # Does not matter in a backtest context
    name='TestPrediction', # Does not matter in a backtest context
    model='xgb',
    quantiles=[0.10, 0.4, 0.50, 0.70, 0.90],
    horizon_minutes=24*60, # TODO: Find out: Does this influence anything? Does this influence which lagged features are available at prediction time?
    resolution_minutes=15,
    forecast_type="demand", # Note, this should become optional
    lat = 1, # should become optional
    lon = 1, # should become optional
    # train_components=False, #should become optional
    # model_type_group=None, # Note, this should become optional
    # hyper_params={}, # Note, this should become optional
    # feature_names=None, # Note, this should become optional
)

# The modelspecs do not do much if only an "id" is specified.
modelspecs = ModelSpecificationDataClass(id=pj['id'])

# Define backtest specs.
backtest_specs = dict(n_folds=3, 
                      # The training horizon also decides for which forecast horizon, backtest forecasts are made.
                      training_horizons=[0.25, 47.0, 24.0])

#### Perform and save the results of the backtest 10 times

In [ ]:
from utils.persisting_artifacts import write_artifacts

In [ ]:
for i in range(10):
    # Perform the backtest
    forecast, models, train_data, validation_data, test_data = train_model_and_forecast_back_test(
        pj,
        modelspecs = modelspecs,
        input_data = input_data,
        **backtest_specs,
    )
    
    # If n_folds > 1, models is a list of models. In that case, only use the first model.
    if backtest_specs['n_folds'] > 1:
        model=models[0]
    else:
        model=models

    run_name = f"{datetime.utcnow():%Y%m%d}_MDM_top_down_sample_{i}"
    write_artifacts(run_name, forecast, model, pj, backtest_specs)

In [ ]:
pd.set_option("display.max_columns", 130)
train_data[0].head()

## Middenmeer forecasts with Vattenfall power forecasts 1 on 1

### Combine EMS measurements, Vattenfall C-ARM data, and predictors to get input data

In [ ]:
# OpenSTEF always expects a column called "load". This is the column it will predict.
load = pd.DataFrame(dict(load=measurements[target_column] - aggregated_vattenfall_load))
input_data = load.merge(predictors, left_index=True, right_index=True, how='inner')

In [ ]:
assert not(input_data.index.duplicated().any()), "There are duplicate indices in the input data."

### Backtest configuration and execution

#### Configure training, prediction, and backtest specifications

In [ ]:
# Define properties of training / prediction. We call this a 'prediction_job'.
pj=PredictionJobDataClass(
    id=1, # Does not matter in a backtest context
    name='TestPrediction', # Does not matter in a backtest context
    model='xgb',
    quantiles=[0.10,0.30,0.50,0.70,0.90],
    horizon_minutes=24*60, # TODO: Find out: Does this influence anything? Does this influence which lagged features are available at prediction time?
    resolution_minutes=15,
    forecast_type="demand", # Note, this should become optional
    lat = 1, # should become optional
    lon = 1, # should become optional
    # train_components=False, #should become optional
    # model_type_group=None, # Note, this should become optional
    # hyper_params={}, # Note, this should become optional
    # feature_names=None, # Note, this should become optional
)

# The modelspecs do not do much if only an "id" is specified.
modelspecs = ModelSpecificationDataClass(id=pj['id'])

# Define backtest specs.
backtest_specs = dict(n_folds=3, 
                      # The training horizon also decides for which forecast horizon, backtest forecasts are made.
                      training_horizons=[0.25, 47.0, 24.0])

#### Perform the backtest
Perform and save the results of the backtest 10 times.

In [ ]:
for i in range(10):
    # Perform the backtest
    forecast, models, train_data, validation_data, test_data = train_model_and_forecast_back_test(
        pj,
        modelspecs = modelspecs,
        input_data = input_data,
        **backtest_specs,
    )

    # Correct forecast and realized for Vattenfall C-ARM measurements and power forecasts
    quantile_columns = [col for col in forecast.columns if "quantile" in col]


    for horizon in set(forecast.horizon):
        forecast.loc[forecast.horizon == horizon, "forecast"] = (
            forecast.query("horizon == @horizon")["forecast"] +
            aggregated_vattenfall_power_forecasts
        )

        forecast.loc[forecast.horizon == horizon, quantile_columns] = (
            forecast.query("horizon == @horizon")[quantile_columns].apply(
                lambda q_col: q_col + aggregated_vattenfall_power_forecasts
            )
        )

        forecast.loc[forecast.horizon == horizon, "realised"] = (
            forecast.query("horizon == @horizon")["realised"] +
            aggregated_vattenfall_load
        )
    
    # If n_folds > 1, models is a list of models. In that case, only use the first model.
    if backtest_specs['n_folds'] > 1:
        model=models[0]
    else:
        model=models
    
    run_name = f"{datetime.utcnow():%Y%m%d}_MDM_with_Vattenfall_bottom_up_sample_{i}"
    write_artifacts(run_name, forecast, model, pj, backtest_specs)

In [ ]:
pd.set_option("display.max_columns", 130)
train_data[0].head()

# Evaluation of the backtest results

## Boxplots of rMAE, rMSE, and rMAE of the 5 percent lowest values

In [ ]:
# Compare results
import openstef.metrics.metrics as metrics

list_res = []
overal_results = {}
for sample in range(10):
    res_metrics = {}
    res_top_down = pd.read_csv(f"./output/20230720_MDM_top_down_sample_{sample}/forecast.csv", parse_dates=True, index_col=0, compression='gzip')
    res_bottom_up = pd.read_csv(f"./output/20230720_MDM_with_Vattenfall_bottom_up_sample_{sample}/forecast.csv", parse_dates=True, index_col=0, compression='gzip')
    res_metrics["Top_down"]= [metrics.r_mae_lowest(res_top_down["realised"], res_top_down["forecast"]),
                              metrics.r_mae(res_top_down["realised"], res_top_down["forecast"]),
                              metrics.rmse(res_top_down["realised"], res_top_down["forecast"])]
    res_metrics["Vattenfall_forecasts_included"] = [metrics.r_mae_lowest(res_bottom_up["realised"], res_bottom_up["forecast"]),
                                                    metrics.r_mae(res_bottom_up["realised"], res_bottom_up["forecast"]),
                                                    metrics.rmse(res_bottom_up["realised"], res_bottom_up["forecast"])]

    res_metrics_df = pd.DataFrame.from_dict(res_metrics)
    res_metrics_df.index = ["rMAE_lowest", "rMAE", "rMSE"]

    overal_results[sample] = res_metrics_df

    list_res.append(overal_results[sample])

results = pd.concat(list_res).reset_index()

fig = results[results["index"]=="rMSE"][["Vattenfall_forecasts_included", "Top_down"]].plot(title="", kind="box", labels=dict(value="rMSE"))
fig.update_layout(yaxis=dict(rangemode='tozero'))
fig.show()

fig = results[results["index"]=="rMAE"][["Vattenfall_forecasts_included", "Top_down"]].plot(title="", kind="box", labels=dict(value="rMAE"))
fig.update_layout(yaxis=dict(rangemode='tozero'))
fig.show()

fig = results[results["index"]=="rMAE_lowest"][["Vattenfall_forecasts_included", "Top_down"]].plot(kind="box",title="", labels=dict(value="rMAE_lowest"))
fig.update_layout(yaxis=dict(rangemode='tozero'))
fig.show()

#### Visualize top_down forecasts

In [ ]:
from utils.quantile_plotting import plot_quantile_forecasts_and_realized

sample = 0
forecasts_top_down = pd.read_csv(f"./output/20230720_MDM_top_down_sample_{sample}/forecast.csv", parse_dates=True, index_col=0, compression='gzip')

horizon = 24.0

plot_quantile_forecasts_and_realized(
    realized=forecasts_top_down.query("horizon == @horizon")["realised"],
    forecast=forecasts_top_down.query("horizon == @horizon")["forecast"],
    quantiles=forecasts_top_down.query("horizon == @horizon")[[q for q in forecasts_top_down.columns if q[:8] == "quantile"]],
    horizon=horizon,
)

#### Visualize forecasts with Vattenfall included

In [ ]:
from utils import quantile_plotting

sample = 0
forecasts_Vattenfall_included = pd.read_csv(f"./output/20230720_MDM_with_Vattenfall_bottom_up_sample_{sample}/forecast.csv", parse_dates=True, index_col=0, compression='gzip')

horizon = 24.0

plot_quantile_forecasts_and_realized(
    realized=forecasts_Vattenfall_included.query("horizon == @horizon")["realised"],
    forecast=forecasts_Vattenfall_included.query("horizon == @horizon")["forecast"],
    quantiles=forecasts_Vattenfall_included.query("horizon == @horizon")[[q for q in forecasts_Vattenfall_included.columns if q[:8] == "quantile"]],
    horizon=horizon,
)

# Export notebook as html
Write this notebook to html.

In [ ]:

nb_fname = '05.Compare_top_down_MDM_to_MDM_with_Vattenfall_1_on_1'
run_name = f"{datetime.utcnow():%Y%m%d}_comparison_top_down_MDM_to_MDM_with_Vattenfall_1_on_1"
command=f"jupyter nbconvert {nb_fname}.ipynb --to html --no-input --output results/{run_name}.html"
print(f"Command to be executed: {command}.")
os.system(command)

# Open points:
- What is the horizon of the customer power forecasts?
    - The power forecasts are updated about 20-35 times per day. For this analysis, the most recent power forecasts are used. So, we can assume the forecast horizon is about one hour.
- Why is the curtailment of the 25th of may not incorporated in the Vattenfall power forecasts?
    - This was a request from TenneT to the Alliander operations to switch of the fields of the Vattenfall windpark. There was no request to Vattenfall.
- How do the customer power forecasts compare to the OpenSTEF forecasts in terms of quality?
- Regarding the 19th of december incident: How long before curtailment, was the curtailment request sent by TenneT?
    - The curtailment request was sent in real-time / just in time.